In [2]:
#tkinter
import tkinter as tk
import tkinter.ttk as ttk
import threading
from PIL import ImageTk, Image
from functools import partial

#selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from urllib.parse import unquote

#other
import time
import json
import os
import import_ipynb

#show image
# from IPython.core.debugger import set_trace
# from IPython.display import Image
# from IPython.core.display import HTML
# from IPython.core.debugger import set_trace
# from IPython.display import clear_output as clear

class Login:
    
    def __init__(self,window,driver): 
#         self.wdsz = wdsz
        self.window = window
        self.driver = driver
        self.main_frame = tk.Frame(self.window)
        self.bg = 'black'
        self.fg = 'orange'
        self.ft = 'Helvetica 12 bold'
        
        self.login_window()
        
    def check_thread_alive(self,thread):
        if thread.is_alive():
            self.window.after(20,self.check_thread_alive,thread)
        else :
            pass
        
    def start_btn_login_check(self,usr,pwd,fr,wd):
        
        th = threading.Thread(target = self.web_login,args = (usr.get(),pwd.get(),fr,wd,))
        th.setDaemon(True)
        th.start()
        self.window.after(20,self.check_thread_alive,th)
        #th.join()
        
#     def login_check(self,usr,pwd,fr,wd):
#         #print('in')
        
#         global var_suc
        
#         #print("usr : " + usr.get())
#         #print("pwd : " + pwd.get())
        
#         chrome_options = Options()
#         chrome_options.add_argument('headless')
        
#         web_login = ComicFuzLogin(webdriver.Chrome(options = chrome_options))
        
#         web_login.get_login_cookie(usr.get(),pwd.get())
#         #login_success = web_login.get_login_cookie(usr.get(),pwd.get())
        
#         login_success = web_login.login()
    
#         #adjust to check
#         if login_success:
#             var_suc.set('Login Successfully.')
#             wd.update()
#             time.sleep(2)
#             fr.destroy()
#         else :
#             var_suc.set('Fail...')
            
#         #print('back')
        
    def login_window(self):
        
#         global window
#         window.title('Login')
#         window.geometry(self.wdsz)
#         window.resizable(0,0)

        self.main_frame.pack(expand = True,fill = 'both')
        self.main_frame.configure(background = self.bg)

        #control size of image
        p = 0.5
        #global font_param
        #font_param = 'Helvetica 12 bold'
        
        #image path
        pre_img = Image.open('Cover.png')
        
        #get old size value
        pre_w , pre_h = pre_img.size[0] , pre_img.size[1]
        
        #resize it
        pre_img = pre_img.resize((int(p * pre_w),int(p * pre_h)),Image.ANTIALIAS)
        img = ImageTk.PhotoImage(pre_img)
        
        print(type(img))
        print(img.width(),img.height())
        
        front_img = tk.Label(self.main_frame,image = img)
        front_img.photo = img    #important line to show image
        front_img.pack()
#         front_img = tk.Canvas(self.main_frame)
#         front_img.create_image(0,0,image = img)
#         front_img.pack(side = 'top')

        #prompt text
        login_title = tk.Label(self.main_frame,text = "It's a program that helps you download your manga.",
                            bg = self.bg,fg = self.fg,height = 2,font = self.ft)
        login_title.pack()

        #entry frame
        btm_frame = tk.Frame(self.main_frame)
        btm_frame.pack(side = 'top',fill = 'y')
        btm_frame.configure(background = self.bg)

        #setting a grid
        usr_text = tk.Label(btm_frame,text = 'mail-address : ',bg = 'dimgrey',fg = self.fg,font = self.ft)
        pwd_text = tk.Label(btm_frame,text = 'password : ',bg = 'dimgrey',fg = self.fg,font = self.ft)


        usr_text.grid(row = 0,sticky = 'nsew')
        pwd_text.grid(row = 1,sticky = 'nsew')

        #btm_frame.grid_columnconfigure(2,weight = 1)
        #btm_frame.grid_rowconfigure(2,weight = 1)

        usr_entry = tk.Entry(btm_frame,font = self.ft)
        pwd_entry = tk.Entry(btm_frame,show = '*',font = self.ft)

        usr_entry.grid(row = 0,column = 1)
        pwd_entry.grid(row = 1,column = 1)

        login_btn = tk.Button(self.main_frame,text = 'submit',bg = self.bg,fg = self.fg,
                           font = self.ft,command = partial(self.start_btn_login_check,usr_entry,pwd_entry,self.main_frame,self.window))
        login_btn.pack(pady = 10)
        
        global var_suc
        var_suc = tk.StringVar()
        
        login_suc_text = tk.Label(self.main_frame,textvariable = var_suc,bg = self.bg,fg = self.fg,height = 2,font = self.ft)
        login_suc_text.pack(expand = True,fill = 'x')
    
    #webdriver
    
    def use_json_login(self):
        login_json_path =  'comicFuz.json'

        if os.path.isfile(login_json_path):
            print("Already login before")
            print("Skip login part\n")
        else :
            print('You haven\'t login yet')
            print('Please login at least once')
            self.get_login_cookie()
        
    def web_login(self,account,password,fr,wd):
        
        progress_window = tk.Toplevel(wd)
        progress_bar = ttk.Progressbar(progress_window,orient='horizontal',length = 100,mode = 'determinate')
        progress_bar.pack()
        
        url = "https://comic-fuz.com/accounts/my_page"
        #driver = webdriver.Chrome()
        self.driver.get(url)
        
        progress_bar['value'] = 30
        wd.update_idletasks()
        
        #user input
        
        #login_success = False
        
        #while login_success == False:
            
        login_wait = WebDriverWait(self.driver,10).until(
            EC.presence_of_element_located((By.XPATH,'/html/body/main/section/div/div/div/section[1]/div/form/div/input'))
        )

        progress_bar['value'] = 50
        wd.update_idletasks()
        
        user_account_block = self.driver.find_element_by_id('account_email')
        user_password_block = self.driver.find_element_by_id('account_password')
        
        print('in login page')

#             user_account = input('Enter your ComicFuz account (email address) : ')
#             user_password = input('Enter your ComicFuz password : ')

        user_account = account
        user_password = password

        user_account_block.clear()
        user_password_block.clear()

        user_account_block.send_keys(user_account)
        user_password_block.send_keys(user_password)
        
        progress_bar['value'] = 75
        wd.update_idletasks()

        self.driver.find_element_by_xpath("/html/body[@class='fuz_accounts_sessions']/main/section[@id='login']/div[@class='inner']/div[@id='main']/div[@class='column_primary']/section[@id='member']/div[@id='sign_in_box']/form[@id='new_account']/div[@class='box_button']/input[@class='btn btn-default button button_decision']").send_keys(Keys.ENTER)
        #time.sleep(2)
        
        global var_suc
        
        try:
            login_finish = WebDriverWait(self.driver,6).until(
                EC.presence_of_element_located((By.XPATH,'/html/body/main/div/div/div/div[1]/div/div/ul/ul[2]/li[1]/a'))
            )
            #some login logic here
            #self.driver.find_element_by_xpath('/html/body/main/div/div/div/div[1]/div/div/ul/ul[2]/li[1]/a')
            print('Login successfully!')
            #login_success = True
            
            time.sleep(2)
            
            progress_bar['value'] = 100
            wd.update_idletasks()

            cookie = self.driver.get_cookies()

            json_cookies = json.dumps(cookie)

            with open('comicFuz.json','w') as f:
                f.write(json_cookies)

            self.driver.quit()
            
            progress_window.destroy()
            
            var_suc.set('Login Successfully.')
            wd.update_idletasks()
            time.sleep(2)
            fr.destroy()
            
            return True
            
        except:
            print('Login failed...')
            print('Please try again!')
            
            var_suc.set('Fail...')
            
            return False

        
window = tk.Tk()
window.title('Login')
window.geometry('500x300')
window.resizable(0,0)

chrome_options = Options()
chrome_options.add_argument('headless')
driver = webdriver.Chrome(options = chrome_options)

#setting new icon
icon = ImageTk.PhotoImage(file = 'Cover.png')
window.tk.call('wm','iconphoto',window._w,icon)
print(icon)

page = Login(window,driver)
l = tk.Label(window,text = "Over",bg = 'black',fg = 'orange',height = 2)
l.pack()

window.mainloop()

importing Jupyter notebook from ListComicCombine.ipynb


NameError: name 'ComicFuz' is not defined